<a href="https://colab.research.google.com/github/a123700/Leo/blob/master/Next_Word_Prediction_with_BI_LSTM_and_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
api_token = {"username":"srhgccc","key":"41c36ccc1ee7bc07f70e459ccbf7df7c"}
import json
import zipfile
import os
 
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')

In [ ]:
!kaggle datasets download -d hsankesara/medium-articles

In [ ]:
!unzip medium-articles.zip

<a name="ilp"></a>
# Import necessary libraries and packages 

In [ ]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

<a name="di"></a>
# Dataset information

**Import Medium-articles-dataset:**

This dataset contains information about randomly chosen medium articles published in 2019 from these 7 publications:

- Towards Data Science
- UX Collective
- The Startup
- The Writing Cooperative
- Data Driven Investor
- Better Humans
- Better Marketing


In [ ]:
medium_data = pd.read_csv('articles.csv')
medium_data.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


Here, we have a **10 different fields and 6508 records** but we will only use **title field** for predicting next word. 

In [ ]:
print("Number of records: ", medium_data.shape[0])
print("Number of fields: ", medium_data.shape[1])

Number of records:  337
Number of fields:  6


<a name="preprocess"></a>
# Display titles of various articles  and preprocess them

<a name="remv"></a>
#### Removing unwanted characters and words in titles

Looking at titles, we can see there are some of unwanted characters and words in it which can not be useful for us to predict infact it might decrease our model accuracy so we have to remove it.

In [ ]:
import re

# Remove non-alphabetic characters (Data Cleaning)
def text_strip(row):
  row = re.sub("(\\t)", " ", str(row)).lower()
  row = re.sub("(\\r)", " ", str(row)).lower()
  row = re.sub("(\\n)", " ", str(row)).lower()

        # Remove _ if it occurs more than one time consecutively
  row = re.sub("(__+)", " ", str(row)).lower()

        # Remove - if it occurs more than one time consecutively
  row = re.sub("(--+)", " ", str(row)).lower()

        # Remove ~ if it occurs more than one time consecutively
  row = re.sub("(~~+)", " ", str(row)).lower()

        # Remove + if it occurs more than one time consecutively
  row = re.sub("(\+\++)", " ", str(row)).lower()

        # Remove . if it occurs more than one time consecutively
  row = re.sub("(\.\.+)", " ", str(row)).lower()

        # Remove the characters - <>()|&©ø"',;?~*!
  row = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", " ", str(row)).lower()

        # Remove mailto:
  row = re.sub("(mailto:)", " ", str(row)).lower()

        # Remove \x9* in text
  row = re.sub(r"(\\x9\d)", " ", str(row)).lower()

        # Replace INC nums to INC_NUM
  row = re.sub("([iI][nN][cC]\d+)", "INC_NUM", str(row)).lower()

        # Replace CM# and CHG# to CM_NUM
  row = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", "CM_NUM", str(row)).lower()

        # Remove punctuations at the end of a word
  row = re.sub("(\.\s+)", " ", str(row)).lower()
  row = re.sub("(\-\s+)", " ", str(row)).lower()
  row = re.sub("(\:\s+)", " ", str(row)).lower()

        # Replace any url to only the domain name
  try:
    url = re.search(r"((https*:\/*)([^\/\s]+))(.[^\s]+)", str(row))
    repl_url = url.group(3)
    row = re.sub(r"((https*:\/*)([^\/\s]+))(.[^\s]+)", repl_url, str(row))
  except:
    pass

        # Remove multiple spaces
  row = re.sub("(\s+)", " ", str(row)).lower()

        # Remove the single character hanging between any two spaces
  row = re.sub("(\s+.\s+)", " ", str(row)).lower()

  return row

In [ ]:
medium_data['text'] = medium_data['text'].apply(lambda x: 'sos '+x+' eos')

medium_data['text'] = medium_data['text'].apply(lambda x: text_strip(x))


In [ ]:
medium_data.sample(5)

,author,claps,reading_time,link,title,text
258,Greg Gascon,368,6,https://medium.com/startup-grind/how-invisible...,How Invisible Interfaces are going to transfor...,sos in the mid-nineties computer scientist at ...
291,Arthur Juliani,9K,6,https://medium.com/emergent-future/simple-rein...,Simple Reinforcement Learning with Tensorflow ...,sos for this tutorial in my reinforcement lear...
182,Christian Hernandez,364,7,https://medium.com/crossing-the-pond/into-the-...,Into the Age of Context – Crossing the Pond – ...,sos spent most of my early career proclaiming ...
300,Milo Spencer-Harper,2.2K,3,https://medium.com/technology-invention-and-mo...,How to build a multi-layered neural network in...,sos in my last blog post thanks to an excellen...
76,Blaise Aguera y Arcas,8.7K,15,https://medium.com/@blaisea/do-algorithms-reve...,Do algorithms reveal sexual orientation or jus...,sos by blaise agüera arcas alexander todorov ...


In [ ]:
data = medium_data['text'].tolist()

print(len(data))

337


<a name="token"></a>
#### Tokenization

Tokenization is the process in which we provide an unique id to all the words and make a word index or we can say vocabulary.

In [ ]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1
vocab_size = 20000  # Only consider the top 20k words
maxlen = 500  # Only consider the first 500 words of each movie review

In [ ]:
print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("machine: ", tokenizer.word_index['machine'])
print("request: ", tokenizer.word_index['request'])

Total number of words:  21968
Word: ID
------------
<oov>:  1
machine:  44
request:  2975


<a name="xy"></a>
# Prepare features and labels

Here, we consider **last element of all sequences as a label**.Then,
We need to perform **onehot encoding on labels corresponding to total_words.**

In [ ]:
from tqdm import tqdm
xs = []
ys = []
forward_words = 200

for line in tqdm(data):
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(len(token_list)-forward_words):
        n_gram_sequence = token_list[i:i+forward_words]
        labels = token_list[i+forward_words]
        xs.append(n_gram_sequence)
        ys.append(labels)

print("\nTotal input sequences: ", len(xs))

100%|██████████| 337/337 [00:03<00:00, 86.52it/s] 


Total input sequences:  542198


In [ ]:
xs = np.array(xs)
ys = np.array(ys)

<a name="train"></a>
# Bi- LSTM Neural Network Model training

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import keras
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Embedding(total_words, 100, input_length=forward_words))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(16, return_sequences = True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(8, return_sequences = False)))
model.add(Dropout(0.1))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(xs, ys, random_state=777, train_size=0.9)

In [ ]:
batch_size = 4096

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [ ]:
import time

epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
# red_lr= ReduceLROnPlateau(monitor='val_accuracy',patience=10,verbose=0,factor=0.1)
# callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]
# history = model.fit(xs, ys, epochs=50, verbose=1, callbacks = [callbacks, red_lr], batch_size=1024, validation_split=0.1)

<a name="acc"></a>
## Plotting Bi- LSTM model accuracy and loss

In [ ]:
# import matplotlib.pyplot as plt

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.legend(['train', 'test'])
# plt.show()

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(['train', 'test'])
# plt.show()

<a name="new"></a>
## Predicting next word of title

In [ ]:
seed_text = "Where"
next_words = 100
  
for _ in tqdm(range(next_words)):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=forward_words, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

<a name="train"></a>
# Transformer Neural Network Model training

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim = 200  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(xs.shape[1],))
embedding_layer = TokenAndPositionEmbedding(forward_words, total_words, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
x = Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
outputs = layers.Dense(total_words, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 200)         4433600   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 200)         348064    
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 200)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 200)               0     

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(xs, ys, random_state=777, train_size=0.9)

In [ ]:
batch_size = 512

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [ ]:
import time

epochs = 30
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
# import keras
# from keras.callbacks import ReduceLROnPlateau
# red_lr= ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=0,factor=0.1)
# callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

# history = model.fit(xs, ys, batch_size=4096, epochs=100, validation_split = 0.15, callbacks = [callbacks, red_lr])

<a name="acc"></a>
## Plotting Transformer model accuracy and loss

In [ ]:
# import matplotlib.pyplot as plt

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.legend(['train', 'test'])
# plt.show()

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(['train', 'test'])
# plt.show()

<a name="new"></a>
## Predicting next word of title

In [ ]:
seed_text = "Mechine learning is"
next_words = 100
  
for _ in tqdm(range(next_words)):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=forward_words, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    if output_word == 'eos':
      break
    seed_text += " " + output_word
print('\n'+seed_text)